<p style="float:right">
<img src="nasa.png" style="display:inline" />
</p>

# Geographical Subsetting with OPeNDAP

For this demonstration, we will examine the [_MEaSUREs Northern Hemisphere Terrestrial Snow Cover Extent Daily 25km EASE-Grid 2.0_](http://nsidc.org/data/docs/measures/nsidc-0530/) dataset from the [NASA MEaSUREs project](https://earthdata.nasa.gov/community/community-data-system-programs/measures-projects). This dataset is managed by [NSIDC DAAC](http://nsidc.org/data/measures/) and made available via, among other methods, the [OPeNDAP](https://en.wikipedia.org/wiki/OPeNDAP) protocol.

### Determining grid coordinates for a geographical area

Let's say that we're interested only in data pertaining to Iceland. We can definte geographical bounds for our area of interest:

In [ ]:
lat_south = 63
lat_north = 67
lon_west = -25
lon_east = -13

As in the notebook _Variable Subsetting with OPeNDAP_, we will use the Python `netCDF4` package's built-in OPeNDAP client to access snow-cover data from June 1, 2012. In this case, we will request only the **latitude** and **longitude** variables:

In [ ]:
import netCDF4

url0 = (
    'http://n5eil01u.ecs.nsidc.org:80/opendap/'
    'MEASURES/NSIDC-0530.001/2012.06.01/nhtsd25e2_20120601_v01r01.nc?'
    'latitude[0:1:719][0:1:719],'
    'longitude[0:1:719][0:1:719]'
)

dataset0 = netCDF4.Dataset(url0)

for v in dataset0.variables:
    print(v, dataset0.variables[v].shape)

Instead of full 720 x 720 grids, we would like to constrain future OPeNDAP requests so that they return data only for the Iceland area. To do so, we'll use NumPy to obtain lists of grid rows and columns where both `latitude` and `longitude` values fall within the defined Iceland area, then take the minima and maxima of each list to construct bounding-box values we can pass to OPeNDAP:

In [ ]:
import numpy as np

lat = dataset0.variables['latitude'][:, :]
lon = dataset0.variables['longitude'][:, :]
row, col = np.where((lat >= lat_south) & (lat <= lat_north) & (lon >= lon_west) & (lon <= lon_east))
bbox = np.min(row), np.max(row), np.min(col), np.max(col)
print(bbox)

Now we can use our bounding-box values as constraints in our OPeNDAP URL to select just the rows and columnsthat correspond to Iceland. The OPeNDAP constraints are given in `lower_bound:stride:upper_bound` form:

In [ ]:
url1 = (
    'http://n5eil01u.ecs.nsidc.org/opendap/'
    'MEASURES/NSIDC-0530.001/2012.06.01/nhtsd25e2_20120601_v01r01.nc?'
    'latitude[453:1:475][310:1:336],'
    'longitude[453:1:475][310:1:336],'
    'merged_snow_cover_extent[0:1:0][453:1:475][310:1:336]'
)

dataset1 = netCDF4.Dataset(url1)

for v in dataset1.variables:
    print(v, dataset1.variables[v].shape)

That's a lot less data!

Finally, we can plot our data on a map of Iceland, using code similar to that shown in the notebook _Variable Subsetting with OPeNDAP_:

In [ ]:
%matplotlib inline
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
iceland_map = Basemap(
    projection='merc',
    resolution='i',
    llcrnrlon=lon_west,
    llcrnrlat=lat_south,
    urcrnrlon=lon_east,
    urcrnrlat=lat_north
)
iceland_map.drawcoastlines()

latitude = dataset1.variables['latitude'][:, :]
longitude = dataset1.variables['longitude'][:, :]
msce = dataset1.variables['merged_snow_cover_extent'][0, :, :]

row_snow, col_snow = np.where((msce >= 10) & (msce <= 16))
x_snow, y_snow = iceland_map(longitude[row_snow, col_snow], latitude[row_snow, col_snow])
iceland_map.scatter(x_snow, y_snow, marker='s', s=400, color='Cyan')

row_ice, col_ice = np.where(msce == 30)
x_ice, y_ice = iceland_map(longitude[row_ice, col_ice], latitude[row_ice, col_ice])
iceland_map.scatter(x_ice, y_ice, marker='s', s=400, color='Blue')